In [17]:
import pandas as pd

# organic
f = pd.ExcelFile(r"C:\Users\user\Desktop\Python\Martech\Python關鍵字分析\new_folder\2_ cathay-ins(2024_08_05-2024_09_01).xlsx")
df = f.parse()
df.columns

Index(['关键词', '点击量', 'Traffic ', '变动', '28 天的体量', '流量变化', '桌面端份额', '移动端份额',
       'KD', '意图', '平均体量', 'CPC', '零点击', '热门网址', 'SERP Features'],
      dtype='object')

In [19]:
df = df.loc[:,['关键词', '点击量', '平均体量']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   关键词     1000 non-null   object
 1   点击量     1000 non-null   object
 2   平均体量    1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [20]:
df.rename(columns={'关键词':"Keyword", '点击量':"Clicks", '平均体量':"Avg_Volume"}, inplace=True)
str_clicks = df["Clicks"].loc[df["Clicks"].apply(lambda x: isinstance(x, str))]
print(str_clicks.unique(),",", str_clicks.count())
str_clicks_zero = str_clicks.loc[str_clicks.apply(lambda x: x == "0")]
print(f"0: {str_clicks_zero.count()}, <50: {586-str_clicks_zero.count()}")

['<50' '0'] , 586
0: 268, <50: 318


In [21]:
df["Clicks"] = df["Clicks"].apply(lambda x: 0 if not isinstance(x, int) else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Keyword     1000 non-null   object
 1   Clicks      1000 non-null   int64 
 2   Avg_Volume  1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [27]:
str_avg = df["Avg_Volume"].loc[df["Avg_Volume"].apply(lambda x: isinstance(x, str))]
print(type(str_avg), str_avg)
print(str_avg.unique(), str_avg.nunique())
df["Avg_Volume"] = df["Avg_Volume"].apply(lambda x: 0 if not isinstance(x, int) else x)
df.info()

<class 'pandas.core.series.Series'> 39     -
54     -
57     -
65     -
73     -
      ..
715    -
717    -
718    -
725    -
727    -
Name: Avg_Volume, Length: 229, dtype: object
['-'] 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Keyword     1000 non-null   object
 1   Clicks      1000 non-null   int64 
 2   Avg_Volume  1000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 23.6+ KB


In [29]:
df["Avg_Volume"] = df["Avg_Volume"].apply(lambda x: float(x))
# df["Avg_Volume"].info()
df["Clicks"] = df["Clicks"].apply(lambda x: float(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Keyword     1000 non-null   object 
 1   Clicks      1000 non-null   float64
 2   Avg_Volume  1000 non-null   float64
dtypes: float64(2), object(1)
memory usage: 23.6+ KB


In [75]:
# CTR (Click Through Rate)
df["CTR"] = (df["Clicks"] / df["Avg_Volume"]).apply(lambda x: round(x*100, 1)) # add "%" later..
df.info()
df.head(2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Keyword     1000 non-null   object 
 1   Clicks      1000 non-null   float64
 2   Avg_Volume  1000 non-null   float64
 3   CTR         939 non-null    float64
dtypes: float64(3), object(1)
memory usage: 31.4+ KB


,Keyword,Clicks,Avg_Volume,CTR
0,國泰 寵物,130.0,1.0,13000.0
1,旅游不便险,110.0,9.0,1222.2


In [76]:
# debug: divided by 0 bug
ctr_zero = df['CTR'].loc[df['Avg_Volume'].apply(lambda x: x == 0.0)]
ctr_zero
df.loc[df['Avg_Volume'].apply(lambda x: x == 0.0), 'CTR'] = 0.0

In [77]:
df.sort_values(by="CTR", ascending=False, inplace=True)
df.head(5)


,Keyword,Clicks,Avg_Volume,CTR
0,國泰 寵物,130.0,1.0,13000.0
1,旅游不便险,110.0,9.0,1222.2
2,旅遊險 不便險,110.0,11.0,1000.0
3,國泰產險活動代碼,320.0,40.0,800.0
4,國泰bobe,280.0,37.0,756.8


In [78]:
df.reset_index(drop=True, inplace=True)
df.head(5)

,Keyword,Clicks,Avg_Volume,CTR
0,國泰 寵物,130.0,1.0,13000.0
1,旅游不便险,110.0,9.0,1222.2
2,旅遊險 不便險,110.0,11.0,1000.0
3,國泰產險活動代碼,320.0,40.0,800.0
4,國泰bobe,280.0,37.0,756.8


In [79]:
kw_list = "國泰"
if isinstance(kw_list, str):
    kw_list = [kw_list] 
    pattern = '|'.join(kw_list)
# print(pattern)

yes_df = df.loc[df['Keyword'].str.contains(pattern, na=False, regex=True)]
no_df = df.loc[~df['Keyword'].str.contains(pattern, na=False, regex=True)]


In [86]:
yes_df.info()
yes_df.reset_index(drop=True, inplace=True)
yes_df.loc[:, "CTR"] = yes_df["CTR"].apply(lambda x: str(x) + '%')
# yes_df.drop("try", axis=1, inplace=True)
yes_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Keyword     211 non-null    object 
 1   Clicks      211 non-null    float64
 2   Avg_Volume  211 non-null    float64
 3   CTR         211 non-null    object 
dtypes: float64(2), object(2)
memory usage: 6.7+ KB


,Keyword,Clicks,Avg_Volume,CTR
0,國泰 寵物,130.0,1.0,13000.0%
1,國泰產險活動代碼,320.0,40.0,800.0%
2,國泰bobe,280.0,37.0,756.8%
3,國泰申根保險,320.0,44.0,727.3%
4,國泰產險客服,4130.0,603.0,684.9%


In [88]:
no_df.info()
no_df.reset_index(drop=True, inplace=True)
no_df.loc[:, "CTR"] = no_df["CTR"].apply(lambda x: str(x) + '%')
no_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Keyword     789 non-null    object 
 1   Clicks      789 non-null    float64
 2   Avg_Volume  789 non-null    float64
 3   CTR         789 non-null    float64
dtypes: float64(3), object(1)
memory usage: 24.8+ KB


,Keyword,Clicks,Avg_Volume,CTR
0,旅游不便险,110.0,9.0,1222.2%
1,旅遊險 不便險,110.0,11.0,1000.0%
2,汽車第三責任險費用,1980.0,407.0,486.5%
3,高意媗,120.0,35.0,342.9%
4,日本滑雪地圖,80.0,24.0,333.3%


In [95]:
def save_to_excel(path, df, brandname):
    try:
        with pd.ExcelWriter(path, mode='a') as writer:
            df.to_excel(writer, sheet_name=brandname, index=False)
    except FileNotFoundError:
        print("No file existed, Creating a new file right away...")
        df.to_excel(path, sheet_name=brandname, index=False)
    else:
        print("Done!")
        

In [96]:
save_to_excel("With_Brandname_CTR.xlsx", yes_df, "國泰")

No file existed, Creating a new file right away...
